In [71]:
import cv2
import numpy as np

img = cv2.imread("forms//49553.png")
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
(thresh, img_bin) = cv2.threshold(img_gray, 188, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
img_bin = cv2.bitwise_not(img_bin)

def remote_line(img,t=0):
#     img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (thresh, img_bin) = cv2.threshold(img_gray, 188, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    img_bin = cv2.bitwise_not(img_bin)
    edges = cv2.Canny(img_bin,30,150,apertureSize = 3)
    if t == 0:
        lines = cv2.HoughLines(edges,1,np.pi/180,250)
        for i in range(len(lines)):
            for rho,theta in lines[i]:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*(a))
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*(a))
            cv2.line(img_bin,(x1,y1),(x2,y2),(0,0,0),3)
        return img_bin
        
# cv2.imshow("img2",remote_line(img))
# cv2.waitKey(0)        

img_rl = remote_line(img)
kernel = np.ones((12,12), np.uint8)
img_dilation = cv2.dilate(img_rl, kernel, iterations=1)
img_dilation = cv2.erode(img_dilation, kernel, iterations=1)

# cv2.imshow("img2",img_dilation)
# cv2.waitKey(0)  

# kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
# connected = cv2.morphologyEx(img_dilation, cv2.MORPH_CLOSE, kernel)
contours, hierarchy = cv2.findContours(image=img_dilation, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_SIMPLE)

max_x = 0
max_y = 0
min_x = 10000
min_y = 10000
(x,y,_) = img.shape
for i in contours:
    for j in range(len(i)):
        if i[j][0][0] > max_x:
            max_x = i[j][0][0]
        if i[j][0][0] < min_x:
            min_x = i[j][0][0]
        if i[j][0][1] > max_y:
            max_y = i[j][0][1]
        if i[j][0][1] < min_y:
            min_y = i[j][0][1]
    if((max_x-min_x)*(max_y-min_y)/(x*y)>= 0.7):
        max_x = 0
        max_y = 0
        min_x = 10000
        min_y = 10000
        continue
    img = cv2.rectangle(img, (min_x,min_y), (max_x,max_y), [0,255,0], 1)
    max_x = 0
    max_y = 0
    min_x = 10000
    min_y = 10000
cv2.imshow("img2",img)
cv2.waitKey(0)

-1